# Prediction of Transcription Factories: A Machine Learning Blueprint

Transcription Factories are those purported sites that have been identified as the breeding grounds for transcripts. Their relevance heightens when the spatial organization of the genome is considered. Since transcription factories attract distant segments of the genome together for inter-mingling and possibly carrying out regulation, true picture emerges of the parts that co-localize and co-express. This has been a major lacuna in the dogma of the current suite of enrichment tools for genomic regions.

## Setting up Target Genomic Regions

In cognizance to [GREG](http://www.dx.doi.org/10.1093/database/baz162), we define genomic regions with a 2 kilobase bin size. These shall be the pivots corresponding to which the features with read counts shall be hinged. Additionally, we want to make our BED file [1-based](http://genome.ucsc.edu/FAQ/FAQformat.html#format1), to align with the format in GREG. 

In [3]:
## Source the UCSC library for Homo sapiens genome, version hg19.
BiocManager::install("BSgenome.Hsapiens.UCSC.hg19")
library(BSgenome.Hsapiens.UCSC.hg19)

Bioconductor version 3.4 (BiocManager 1.30.10), R 3.3.3 (2017-03-06)

Installing package(s) 'BiocVersion', 'BSgenome.Hsapiens.UCSC.hg19'

installing the source package ‘BSgenome.Hsapiens.UCSC.hg19’


Old packages: 'Hmisc', 'KernSmooth', 'Matrix', 'cluster', 'covr', 'dplyr',
  'foreign', 'fs', 'ggplot2', 'lifecycle', 'mgcv', 'plyr', 'rlang', 'servr',
  'testthat', 'vctrs'

Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, cbind, colnames, do.call

From the chromosome size definitions available in the loaded package, we create an index as follows. This shall be the premises on which our genomic bins shall be carved out.

In [4]:
si <- seqinfo(BSgenome.Hsapiens.UCSC.hg19)
si

Seqinfo object with 93 sequences (1 circular) from hg19 genome:
  seqnames       seqlengths isCircular genome
  chr1            249250621      FALSE   hg19
  chr2            243199373      FALSE   hg19
  chr3            198022430      FALSE   hg19
  chr4            191154276      FALSE   hg19
  chr5            180915260      FALSE   hg19
  ...                   ...        ...    ...
  chrUn_gl000245      36651      FALSE   hg19
  chrUn_gl000246      38154      FALSE   hg19
  chrUn_gl000247      36422      FALSE   hg19
  chrUn_gl000248      39786      FALSE   hg19
  chrUn_gl000249      38502      FALSE   hg19

The function **tileGenome** allows to structure a definitive GenomicRanges object.

In [5]:
Bins2k <-  tileGenome(si, tilewidth = 2000, cut.last.tile.in.chrom = TRUE)
Bins2k

GRanges object with 1568625 ranges and 0 metadata columns:
                  seqnames         ranges strand
                     <Rle>      <IRanges>  <Rle>
        [1]           chr1  [   1,  2000]      *
        [2]           chr1  [2001,  4000]      *
        [3]           chr1  [4001,  6000]      *
        [4]           chr1  [6001,  8000]      *
        [5]           chr1  [8001, 10000]      *
        ...            ...            ...    ...
  [1568621] chrUn_gl000249 [30001, 32000]      *
  [1568622] chrUn_gl000249 [32001, 34000]      *
  [1568623] chrUn_gl000249 [34001, 36000]      *
  [1568624] chrUn_gl000249 [36001, 38000]      *
  [1568625] chrUn_gl000249 [38001, 38502]      *
  -------
  seqinfo: 93 sequences (1 circular) from hg19 genome

In [32]:
targetBED <- as.data.frame(Bins2k)
targetBED <- targetBED[, 1:3]
colnames(targetBED) <- c("Chrom", "Start", "End")
head(targetBED)

,Chrom,Start,End
,<fct>,<int>,<int>
1,chr1,1,2000
2,chr1,2001,4000
3,chr1,4001,6000
4,chr1,6001,8000
5,chr1,8001,10000
6,chr1,10001,12000


These are the regions where the read counts need to be mapped.

## Feature Files

For exemplifying, let us pick up the ChIP-Seq (BAM file: a replicate) for epigenetic mark - H3K4me1 for the H1 hESC cell line.(WIP; More instances to follow)

![](siftingBAM.jpg)

The BAM file is indexed first.

In [6]:
system("samtools index -b ./sampleDataH1/H3K4me1/ENCFF441KOL.bam")

Now that we have the index files along, we proceed towards getting read counts for our defined bin-size.  We achieve this via deeptools suite, a function called bamCoverage. 

In [7]:
system("bamCoverage --bam ./sampleDataH1/H3K4me1/ENCFF441KOL.bam -o ./sampleDataH1/H3K4me1/ENCFF441KOL_2Kb.bw --binSize 2000 --normalizeUsing BPM --effectiveGenomeSize 2913022398 --outFileFormat bedgraph --maxFragmentLength 36")

In [13]:
## Calling appropriate libraries.

library(GenomicRanges)
library(rtracklayer)
library(IRanges)

In [18]:
## Loading the read count data

sample <- import.bed("./sampleDataH1/H3K4me1/ENCFF441KOL_2Kb.bw")
sample

GRanges object with 1218531 ranges and 1 metadata column:
                    seqnames           ranges strand |        name
                       <Rle>        <IRanges>  <Rle> | <character>
        [1]             chr1   [    1,  8000]      * |           0
        [2]             chr1   [ 8001, 12000]      * |    0.113715
        [3]             chr1   [12001, 16000]      * |           0
        [4]             chr1   [16001, 18000]      * |    0.113715
        [5]             chr1   [18001, 40000]      * |           0
        ...              ...              ...    ... .         ...
  [1218527] chrUn_GL000218v1 [ 42001, 126000]      * |           0
  [1218528] chrUn_GL000218v1 [126001, 128000]      * |    0.113715
  [1218529] chrUn_GL000218v1 [128001, 160000]      * |           0
  [1218530] chrUn_GL000218v1 [160001, 161147]      * |    0.113715
  [1218531]           chrEBV [     1, 171823]      * |           0
  -------
  seqinfo: 195 sequences from an unspecified genome; no seqle

One of the issues of using *bamCoverage* is that it merges the adjacent bins if the reads overlap. Since, we mandatorily require the reads to fall into a prespecified interval (Bins2K object), we shall have to look for and map the overlaps.

In [33]:
sourceBED <- as.data.frame(sample)
sourceBED <- sourceBED[, c(1:3,6)]
colnames(sourceBED) <- c("Chrom", "Start", "End", "Score")
head(sourceBED)

,Chrom,Start,End,Score
,<fct>,<int>,<int>,<chr>
1,chr1,1,8000,0
2,chr1,8001,12000,0.113715
3,chr1,12001,16000,0
4,chr1,16001,18000,0.113715
5,chr1,18001,40000,0
6,chr1,40001,42000,0.113715


In [30]:
mappingReadsBins <- function(targetBED, sourceBED)
{
  sourceBED$Chrom <- factor(sourceBED$Chrom, levels=levels(targetBED$Chrom)) # match chromosomes for consistency 
  # names in random binned file to the fixed bins file.
      for(i in 1:nrow(sourceBED))
      {
        for(j in 1:nrow(targetBED))
          {
            if(sourceBED$Chrom[i] == targetBED$Chrom[j] && sourceBED$Start[i] <= targetBED$Start[j] && sourceBED$End[i] >= targetBED$End[j])
            {
              targetBED$Score[j] <- sourceBED$Score[i]
            }
          }
      }
  return(targetBED)
      }

In [31]:
sample2k <- mappingReadsBins(targetBED, sourceBED)
sample2k